# 洋流資料分析 with Mplleaflet and Folium
https://pypi.python.org/pypi/folium  
https://github.com/python-visualization/folium  
https://folium.readthedocs.io/en/latest/  
https://ocefpaf.github.io/python4oceanographers/blog/2015/11/16/folium_quiver/  
http://www.digital-geography.com/python-and-webmaps-folium/#.V6w07Pl96T8   

## MySQL 相關

### 啟動 MySQL Docker container
https://hub.docker.com/_/mysql/  
docker run -d --name=mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=ubuntu armv7/armhf-mysql:5.5.44

In [1]:
# if need to reset MySQL password: 
#     https://dev.mysql.com/doc/refman/5.5/en/resetting-permissions.html
#     https://dev.mysql.com/doc/mysql-windows-excerpt/5.7/en/resetting-permissions-windows.html
# http://stackoverflow.com/questions/35684400/how-to-use-python-3-5-1-with-a-mysql-dtabase

import pymysql

def get_sql_connection(host='localhost', passwd='',db=''):
    return pymysql.connect(host=host,
                           port=3306,
                           user='root',
                           passwd=passwd,
                           charset='UTF8',
                           db=db)


def read_from_mysql(host='localhost', passwd='',db='',table = ''):
    conn = get_sql_connection(host = host,
                              passwd = passwd,
                              db = db)
    
    sql = 'SELECT * FROM {0}'.format(table)
    return pd.read_sql(sql, conn)


def write_to_mysql(df = None, host='localhost', passwd='',db='',table = ''):
    con = get_sql_connection(host = host,
                             passwd = passwd,
                             db = db)
    df.to_sql(name = table,
              con = con, 
              flavor = 'mysql',
              if_exists = 'append')

In [2]:
# read_from_mysql(host='netbrain.noip.me', passwd='',db='location',table = '')
# df = read_from_mysql(host='localhost', passwd='Wei1234c',db='test',table = 'items')
# df

In [3]:
# write_to_mysql(df, host='netbrain.noip.me', passwd='ubuntu', db='test', table = 'data')

## 處理洋流資料

In [4]:
%pylab inline
import pandas as pd
import folium
import mplleaflet

Populating the interactive namespace from numpy and matplotlib


In [5]:
def read_data_file(file):

    with open(file) as f:
        df = pd.read_csv(f,  
                         delim_whitespace = True,
                         comment = '%', 
                         header = None
                        )
    
    # index 名稱
    df.index.name = 'No' 

    # columns 名稱
    columns = ['Longitude', 'Latitude', 'U_comp', 'V_comp', 'VectorFlag', 'U_StdDev', 'V_StdDev', 
               'Covariance', 'StdDev', 'X_Distance', 'Y_Distance', 'Range', 'Bearing', 'Velocity', 'Direction']    
    site_contributors_count = 16
    columns.extend(['Site_Contributor_{0}'.format(i) for i in range(1, site_contributors_count + 1)])
    df.columns = columns
    
    return df

In [6]:
file = 'TOTL_TORO_2015W01.tuv'
df = read_data_file(file)
df.tail()

,Longitude,Latitude,U_comp,V_comp,VectorFlag,U_StdDev,V_StdDev,Covariance,StdDev,X_Distance,...,Site_Contributor_7,Site_Contributor_8,Site_Contributor_9,Site_Contributor_10,Site_Contributor_11,Site_Contributor_12,Site_Contributor_13,Site_Contributor_14,Site_Contributor_15,Site_Contributor_16
No,,,,,,,,,,,,,,,,,,,,,
1373,123.058326,23.646010,0.041,-4.743,0,3.995,16.822,-27.878,16.584,210.0,...,0,0,1065,0,0,0,0,0,0,0
1374,123.059742,23.736275,4.844,-3.624,0,3.717,14.509,-1.222,15.383,210.0,...,0,0,1230,0,0,0,0,0,0,0
1375,123.061165,23.826538,12.426,-7.573,0,4.589,17.328,66.957,14.221,210.0,...,0,0,1125,0,0,0,0,0,0,0
1376,123.062592,23.916801,18.365,-18.871,0,5.153,19.955,98.654,13.645,210.0,...,0,0,1029,0,0,0,0,0,0,0
1377,123.064025,24.007062,22.041,-14.520,0,6.032,20.290,171.829,16.860,210.0,...,0,0,904,0,0,0,0,0,0,0


In [7]:
def do_mplleaflet(lons, lats, u, v, 
                  sub = 1, scale = 1000,
                  color = 'deepskyblue',
                  alpha = 0.8, 
                  file = 'mplleaflet_map.html',
                  tiles = 'esri_aerial'):
    
    # 描繪各點向量
    ax.quiver(lons[::sub], lats[::sub], u[::sub], v[::sub], 
              color = color, alpha = alpha, scale = scale)
    
    # 將 matplotlib figure 輸出成 GeoJSON
    gj = mplleaflet.fig_to_geojson(fig = fig) 
    
    # 存檔 (html)
    mplleaflet.save_html(fig = fig, fileobj = file, tiles = tiles)
#     mplleaflet.display(fig = fig, tiles = 'esri_aerial')
    
    return gj

In [8]:
def get_quiver_marker_layer(geojson):
    
    feature_group = folium.FeatureGroup(name='quiver')

    # 使用 HTML5 SVG 描繪各點的向量，並嵌入在 Folium 的 DivIcon 物件中
    for feature in geojson['features']:
        if feature['geometry']['type'] == 'Point':
            lon, lat = feature['geometry']['coordinates']
            div = feature['properties']['html']

            icon_anchor = (feature['properties']['anchor_x'],
                           feature['properties']['anchor_y'])

            icon = folium.features.DivIcon(html = div, icon_anchor = icon_anchor)
            marker = folium.Marker([lat, lon], icon = icon)
            feature_group.add_children(marker)
        else:
            msg = "Unexpected geometry {}".format
            raise ValueError(msg(feature['geometry']))

    return feature_group

In [9]:
def get_WMS_tile_layer(url = 'http://gmrt.marine-geo.org/cgi-bin/mapserv?map=/public/mgg/web/gmrt.marine-geo.org/htdocs/services/map/wms_merc.map'):   
    
    # 抓取並製作 WMS 圖層
    wms = folium.features.WmsTileLayer(url, name='GMRT',
                                       format='image/png',
                                       layers='topo',
                                       transparent=True)
    
    feature_group = folium.FeatureGroup(name='Topo')
    feature_group.add_children(wms)
    
    return feature_group

In [10]:
if __name__ == '__main__':
    
    # parse 原始資料
    file = 'TOTL_TORO_2015W01.tuv'
    df = read_data_file(file)
    
    # 寫到 db
#     write_to_mysql(df, host='netbrain.noip.me', passwd='ubuntu', db='test', table = 'data')
    
    # 由 db 讀出
#     df = read_from_mysql(host='netbrain.noip.me', passwd='ubuntu',db='test',table = 'data')
    
    # 用 mplleaflet 繪製各點向量 (藍色)
    output_file = 'mplleaflet_map.html'
    fig, ax = plt.subplots()    
    gj = do_mplleaflet(df['Longitude'], df['Latitude'], df['U_comp'], df['V_comp'], file = output_file)
    
    
    
    
    # 用 mplleaflet 繪製各點向量 (黑色) 
    output_file = 'folium_map.html'
    fig, ax = plt.subplots()    
    gj = do_mplleaflet(df['Longitude'], df['Latitude'], df['U_comp'], df['V_comp'], file = output_file, color = 'black')
    
    # 製作圖層
    quiver_layer = get_quiver_marker_layer(gj)    
    WMS_tile_layer = get_WMS_tile_layer()
    
    # 建立 Folium map
    mapOBJ = folium.Map(location = [df.Latitude.mean(), df.Longitude.mean()], tiles = "OpenStreetMap", zoom_start = 7)

    # 嵌入圖層
    mapOBJ.add_children(WMS_tile_layer)
    mapOBJ.add_children(quiver_layer)
    mapOBJ.add_children(folium.map.LayerControl())

    mapOBJ.save(output_file)    